# Density Prediction

Notebook to use the Murphy et al. [2024] density model to generate predictions. 

[Model Fitting](./RF_model.ipynb) is done here and saved into a pickle file. 


In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.gridspec import GridSpec

import cartopy.crs as ccrs
from cartopy.feature.nightshade import Nightshade

In [2]:
pth = '../'
sys.path.append(os.path.dirname(pth))

In [3]:
from mltdm.rf_den import rf_den

In [4]:
# define the path to the model
mdl_sv = r"D:\data\SatDensities\FI_GEO_RFdat_AIMFAHR.pkl"

# initiate the random forest density object
# this sets up the grids for predicting density
# two grids are intitated
# 1) MLT grid
# 2) Geographic grid
# When initiating you can also change the 
# grid dimension by specifing
# n_lat - number of latitude bins
# n_mlt - number of magnetic local time bins
rf_mod = rf_den(input_f=mdl_sv)


In [5]:
import pandas as pd
pd.read_pickle(mdl_sv) 

(RandomForestRegressor(max_features=0.5, min_samples_leaf=5, n_estimators=500,
                       n_jobs=10, oob_score=True, random_state=17),
          1300_02  43000_09  85550_13  94400_18  SYM_H index     AE    SatLat  \
 200511  7.490862  9.911153  9.455465  9.228625        -15.0  105.0 -60.03617   
 242348  7.465065  9.924064  9.463639  9.232483         -6.0   18.0 -60.03854   
 994822  7.696125  9.986703  9.501317  9.272544          5.0   22.0 -87.11732   
 779234  6.966885  9.780912  9.396972  9.171520        -10.0   78.0  25.74049   
 623312  6.596638  9.738198  9.379986  9.153376          3.0   19.0 -20.07795   
 ...          ...       ...       ...       ...          ...    ...       ...   
 172003  7.550780  9.954000  9.479843  9.246430        -20.0  301.0 -33.99006   
 952506  7.523355  9.920227  9.458151  9.237336          2.0   28.0 -12.79244   
 542667  6.572343  9.752442  9.385822  9.157213        -12.0   56.0 -15.53038   
 346251  7.313595  9.831315  9.418499  9.18

# Polar MLT plots

In [ ]:
# predict density on the mlt grid
# you can predict density for a single time point over the entire grid
# or a date range over the entire grid
denMLT = rf_mod.pred_den_mlt(sdate='2003-01-01 00:00', edate='2003-01-01 00:20')

# plot the denisty
# if no date is passed the first date is plotted
rf_mod.plot_dpolar(denMLT,cmap=plt.cm.inferno)


In [ ]:
# plot the density for a specifc time from the previously prediction call
# use vmin and vmax to set the color scale range 
# and change the color map
rf_mod.plot_dpolar(denMLT,cmap=plt.cm.plasma, date='2003-01-01 00:15', vmin=2E-12, vmax=6E-12)

# Geographic Plots

In [ ]:
# predict density on a geographic grid
# over a 3 day period

denGeo = rf_mod.pred_den_geo(sdate='2003-01-01 00:00:00', edate='2003-01-03 00:00:00')

In [ ]:
# plot the density at specific time on the created figure
fig2 = plt.figure()
ax2 = fig2.add_subplot(111,projection=ccrs.PlateCarree())
rf_mod.plot_dgeo(denGeo, ax=ax2, date='2003-01-01 06:00:00')

In [ ]:
fig2 = plt.figure()
ax2 = fig2.add_subplot(111,projection=ccrs.PlateCarree())
rf_mod.plot_dgeo(denGeo, ax=ax2, date='2003-01-01 06:00:00')

# Combine Plots using Grid Spec

In [ ]:
date = '2003-01-01 00:05:00'
vmin = 2E-12
vmax = 5E-12

fig = plt.figure(figsize=(10,4))

gs = GridSpec(1, 2, width_ratios=[1, 2])
ax1 = fig.add_subplot(gs[0], projection='polar')
ax2 = fig.add_subplot(gs[1], projection=ccrs.PlateCarree())

pol_im = rf_mod.plot_dpolar(denMLT, ax=ax1,date=date,vmin=vmin,vmax=vmax)
geo_im = rf_mod.plot_dgeo(denGeo, ax=ax2,date=date,vmin=vmin,vmax=vmax)

fig.delaxes(fig.axes[2])
fig.delaxes(fig.axes[2])

fig.colorbar(geo_im, ax=ax2, orientation='horizontal', label='Neutral Density')
